<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#import库" data-toc-modified-id="import库-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>import库</a></span></li><li><span><a href="#导入数据" data-toc-modified-id="导入数据-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>导入数据</a></span></li><li><span><a href="#导入bert" data-toc-modified-id="导入bert-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>导入bert</a></span></li><li><span><a href="#数据处理" data-toc-modified-id="数据处理-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>数据处理</a></span></li><li><span><a href="#模型定义" data-toc-modified-id="模型定义-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>模型定义</a></span></li><li><span><a href="#测试模型" data-toc-modified-id="测试模型-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>测试模型</a></span></li></ul></div>

# import库

In [1]:
import os
from tqdm import tqdm
import random
import time
import torch
from torch import nn
import torch.utils.data as Data
import  torch.nn.functional as F
import sys
import numpy as np
import pandas as pd 

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(torch.__version__, device)

1.9.1+cu111 cuda


# 导入数据

In [2]:
def loadfile():
    neg=pd.read_excel('D:/学习/研一下/大数据概论/neg_30000.xls',header=None)
    pos=pd.read_excel('D:/学习/研一下/大数据概论/pos_30000.xls',header=None)

    combined=np.concatenate((pos[0], neg[0]))
    # print(type(pos[0][0])) <str>
    y = np.concatenate((np.ones(len(pos),dtype=int), np.zeros(len(neg),dtype=int)))
    # pos 1, neg 0

    return combined, y

In [3]:
#构造数据
def data_classfier():
    combined,y = loadfile()
    data = combined
    labels = y
    print('Shape of data tensor:', len(data))
    print('Shape of label tensor:', len(labels))
    


    indices = np.arange(data.shape[0])
    np.random.shuffle(indices)  #打乱
    data = data[indices]

    labels = labels[indices]
    
    VALIDATION_SPLIT = 0.2
    nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])
    x_train = data[:-nb_validation_samples]
    y_train = labels[:-nb_validation_samples]
    x_val = data[-nb_validation_samples:]
    y_val = labels[-nb_validation_samples:]
    return x_train,y_train,x_val,y_val

In [4]:
x_train,y_train,x_val,y_val = data_classfier()

Shape of data tensor: 60000
Shape of label tensor: 60000


In [5]:
print(type(x_train))
print(type(y_train))
print(type(x_val))
print(type(y_val))
print(len(x_train))
print(len(y_train))
print(len(x_val))
print(len(y_val))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
48000
48000
12000
12000


In [6]:
print(x_train[:5])

["I originally went to Dr Shivananjappa's office after reading all of the reviews from acne patients as I have been struggling with adult acne.  The first time I called the office to make an appt. she actually got on the phone with me as I explained my issues and asked many questions about previous remedies and procedures I had tried over the years.  She spent almost an hour on the phone with me and was so personable yet very professional sharing with me what was possible and what may not be, etc.  I was very impressed with her after meeting for my first appointment and ended up staying after it chatting with her for over 45 mins about other things (like pets!).  She is a very warm yet professional person that made me feel at ease.  I actually ended up going back for a coolsculpt procedure and I simply cannot say enough about this wonderful person as well as how pleased I am with my results.  I'm lucky I live very close to her office but I would encourage anyone considering a procedure

In [7]:
print(y_train[:5])

[1 0 0 0 1]


# 导入bert

In [8]:
from transformers import BertModel, BertTokenizer
# 这里我们调用bert-base模型，同时模型的词典经过小写处理
model_name = 'bert-base-uncased'
# 读取模型对应的tokenizer
tokenizer = BertTokenizer.from_pretrained(model_name, cache_dir='./transformers/bert-base-uncased/')
# 载入模型
model = BertModel.from_pretrained(model_name, cache_dir='./transformers/bert-base-uncased/')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
model.to(device)
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

# 数据处理

In [10]:
train_data = []
for i in  range(len(x_train)):
    train_data.append([])
    train_data[i].append(x_train[i])
    train_data[i].append(y_train[i])

In [11]:
print(train_data[:5])

[["I originally went to Dr Shivananjappa's office after reading all of the reviews from acne patients as I have been struggling with adult acne.  The first time I called the office to make an appt. she actually got on the phone with me as I explained my issues and asked many questions about previous remedies and procedures I had tried over the years.  She spent almost an hour on the phone with me and was so personable yet very professional sharing with me what was possible and what may not be, etc.  I was very impressed with her after meeting for my first appointment and ended up staying after it chatting with her for over 45 mins about other things (like pets!).  She is a very warm yet professional person that made me feel at ease.  I actually ended up going back for a coolsculpt procedure and I simply cannot say enough about this wonderful person as well as how pleased I am with my results.  I'm lucky I live very close to her office but I would encourage anyone considering a procedur

In [12]:
test_data = []
for i in  range(len(x_val)):
    test_data.append([])
    test_data[i].append(x_val[i])
    test_data[i].append(y_val[i])

In [13]:
print(test_data[:5])

[['I have been visiting Aesthetica Med Spa for all  my skin needs for years- with wonderful results! Kim is fabulous with Botox injections, a true artist, and Christina has made my face youthful and glowing again. I highly recommend Aesthetica for their team of skilled professionals and reasonable prices- get on their mailing list and you will receive information of monthly specials.', 1], ["Absolutely Amazing! I don't have words for how professional and intelligent and talented and kind the staff and Dr. Loftus was. My dad went in for surgery this morning the doctor was finished in an hour and he was more that accommodating to my family and myself. The Nurse Practitioner Michelle Gregory went out of her way to personally write and sign doctors notes for myself and my siblings for our jobs, with no questions asked. I would highly highly recommend this practice!", 1], ['Avoid: doctor cares more for his financial well being than your well being I never thought I\'d write a negative revie

In [14]:
def pretreatment(original_data):
    i = 0
    for element in tqdm(original_data):
        temporary = []
        original_data[i][0] = torch.tensor(tokenizer.encode(element[0], add_special_tokens=True, max_length = 512, padding='max_length', truncation=True))
        temporary.append(element[1])
        original_data[i][1] = torch.tensor(temporary)
        i = i+1
    features =  torch.cat([original_data[i][0].unsqueeze(0).long() for i in range(len(original_data))])
    labels =  torch.cat( [original_data[i][1].long() for i in range(len(original_data))], 0)
    return features, labels

In [15]:
train_set = Data.TensorDataset(*(pretreatment(train_data)))
test_set = Data.TensorDataset(*(pretreatment(test_data)))

100%|████████████████████████████████████████████████████████████████████████| 12000/12000 [00:49<00:00, 243.06it/s]


In [16]:
batch_size = 2
train_iter = Data.DataLoader(train_set, batch_size)# , shuffle=True)
test_iter = Data.DataLoader(test_set, batch_size)# , shuffle=True)

# 模型定义

In [17]:
class GlobalMaxPool1d(nn.Module):
    def __init__(self):
        super(GlobalMaxPool1d, self).__init__()
    def forward(self, x):
         # x shape: (batch_size, channel, seq_len)
        return F.max_pool1d(x, kernel_size=x.shape[2]) # shape: (batch_size, channel, 1)

In [18]:
class TextCNN(nn.Module):
    def __init__(self, embed_size, kernel_sizes, num_channels):
        super(TextCNN, self).__init__()
        # self.embedding = nn.Embedding(len(vocab), embed_size)
        # 不参与训练的嵌入层
        # self.constant_embedding = nn.Embedding(len(vocab), embed_size)
        self.dropout = nn.Dropout(0.5)
        self.decoder_1 = nn.Linear(sum(num_channels), 256)
        self.decoder_2 = nn.Linear(256, 2)
        # 时序最大池化层没有权重，所以可以共用一个实例
        self.pool = GlobalMaxPool1d()
        self.convs = nn.ModuleList()  # 创建多个一维卷积层
        
        for c, k in zip(num_channels, kernel_sizes):
            self.convs.append(nn.Conv1d(in_channels = embed_size, 
                                        out_channels = c, 
                                        kernel_size = k))

    def forward(self, inputs):
        outputs = model(inputs)[0] #shape（512， 768）
        embeddings = outputs # (batch, seq_len, embed_size)
        # 根据Conv1D要求的输入格式，将词向量维，即一维卷积层的通道维(即词向量那一维)，变换到前一维
        embeddings = embeddings.permute(0, 2, 1) # 交换维度的函数
        # 对于每个一维卷积层，在时序最大池化后会得到一个形状为(批量大小, 通道大小, 1)的
        # Tensor。使用flatten函数去掉最后一维，然后在通道维上连结
        encoding = torch.cat([self.pool(F.relu(conv(embeddings))).squeeze(-1) for conv in self.convs], dim=1)
        # 应用丢弃法后使用全连接层得到输出
        middle = self.decoder_1(self.dropout(encoding))
        
        outputs = self.decoder_2(self.dropout(F.relu(middle)))
        return outputs

In [19]:
embed_size, kernel_sizes, nums_channels = 768, [3, 4, 5], [100, 100, 100]
net = TextCNN(embed_size, kernel_sizes, nums_channels)

In [20]:
def evaluate_accuracy(data_iter, net, device=None):
    if device is None and isinstance(net, torch.nn.Module):
        # 如果没指定device就使用net的device
        device = list(net.parameters())[0].device 
    acc_sum, n = 0.0, 0
    with torch.no_grad():
        for X, y in tqdm(data_iter, desc ='evaluate'):
            if isinstance(net, torch.nn.Module):
                net.eval() # 评估模式, 这会关闭dropout
                acc_sum += (net(X.to(device)).argmax(dim=1) == y.to(device)).float().sum().cpu().item()
                net.train() # 改回训练模式
            else: # 自定义的模型, 3.13节之后不会用到, 不考虑GPU
                if('is_training' in net.__code__.co_varnames): # 如果有is_training这个参数
                    # 将is_training设置成False
                    acc_sum += (net(X, is_training=False).argmax(dim=1) == y).float().sum().item() 
                else:
                    acc_sum += (net(X).argmax(dim=1) == y).float().sum().item() 
            n += y.shape[0]
    return acc_sum / n

In [21]:
def train(train_iter, test_iter, net, loss, optimizer, device, num_epochs):
    net = net.to(device)
    print("training on ", device)
    batch_count = 0
    best_acc = 0
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        print(time.strftime("%Y-%m-%d \t %H:%M:%S", time.localtime()), end='\t')
        for X, y in tqdm(train_iter, desc ='train'):
            X = X.to(device)
            y = y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y) 
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            train_l_sum += l.cpu().item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().cpu().item()
            n += y.shape[0]
            batch_count += 1
        test_acc = evaluate_accuracy(test_iter, net)
        if test_acc > best_acc:
            best_acc = test_acc
            print("saving", end='\t')
            PATH = "./SaveModel/BertBaseEpoch10_yelp_" + str(best_acc) + ".pth"
            torch.save(net, PATH) 
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, time %.1f sec'
              % (epoch + 1, train_l_sum / batch_count, train_acc_sum / n, test_acc, time.time() - start))

In [22]:
lr, num_epochs = 0.001, 10
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, net.parameters()), lr=lr)
loss = nn.CrossEntropyLoss()
train(train_iter, test_iter, net, loss, optimizer, device, num_epochs)

training on  cuda
2022-03-11 	 20:09:34	

evaluate: 100%|█████████████████████████████████████████████████████████████████| 6000/6000 [20:24<00:00,  4.90it/s]


saving	epoch 1, loss 0.2184, train acc 0.929, test acc 0.962, time 14125.2 sec
2022-03-12 	 00:04:59	

evaluate: 100%|█████████████████████████████████████████████████████████████████| 6000/6000 [20:23<00:00,  4.90it/s]


saving	epoch 2, loss 0.0888, train acc 0.945, test acc 0.964, time 14104.9 sec
2022-03-12 	 04:00:04	

...

epoch 4, loss 0.0420, train acc 0.951, test acc 0.965, time 14248.5 sec
2022-03-12 	 11:52:04	

evaluate: 100%|█████████████████████████████████████████████████████████████████| 6000/6000 [21:27<00:00,  4.66it/s]


epoch 5, loss 0.0325, train acc 0.952, test acc 0.963, time 14311.1 sec
2022-03-12 	 15:50:35	

train:   3%|█▋                                                                | 618/24000 [05:44<3:37:20,  1.79it/s]


KeyboardInterrupt: 

# 测试模型

这个使用的权重不是保存的最好权重而是最后训练的权重

In [23]:
def predict_sentiment(net, sentence):
    """sentence是词语的列表"""
    device = list(net.parameters())[0].device
    sentence = torch.tensor(tokenizer.encode(s, add_special_tokens=True, max_length = 512, padding='max_length', truncation=True), device=device)
    result = net(sentence.view((1, -1)))
    m = nn.Softmax(dim=1)
    print(m(result))
    label = torch.argmax(result, dim=1)
    return 'positive' if label.item() == 1 else 'negative'

In [24]:
print("请输入一句与医疗相关的英文：")
s = input() 
print(predict_sentiment(net, s))

请输入一句与医疗相关的英文：
I feel that this hospital is average and the charges are high, but the medical level is average
tensor([[0.5096, 0.4904]], device='cuda:0', grad_fn=<SoftmaxBackward>)
negative


In [25]:
print("请输入一句与医疗相关的英文：")
s = input() 
print(predict_sentiment(net, s))

请输入一句与医疗相关的英文：
This hospital is very good. The dentist in charge of me is very patient and the treatment effect is good
tensor([[0.2349, 0.7651]], device='cuda:0', grad_fn=<SoftmaxBackward>)
positive


In [26]:
print("请输入一句与医疗相关的英文：")
s = input() 
print(predict_sentiment(net, s))

请输入一句与医疗相关的英文：
It's too bad. It was just a minor illness. It hasn't been ranked in the number. It's getting worse and worse
tensor([[9.9999e-01, 6.0150e-06]], device='cuda:0', grad_fn=<SoftmaxBackward>)
negative
